In [3]:
import os
import csv
import pandas as pd
import matplotlib.pyplot as plt
import gensim
import numpy as np

from gensim.models import CoherenceModel, LdaModel, LsiModel, HdpModel
from gensim.models.wrappers import LdaMallet
from gensim.corpora import Dictionary
import pyLDAvis.gensim

import os, re, operator, warnings
warnings.filterwarnings('ignore')  # Let's not pay heed to them right now
%matplotlib inline

C:\Users\agodbole\Anaconda3\lib\site-packages\gensim\utils.py:1209: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")


In [4]:
import pickle

In [5]:
payday_hdpmodel=HdpModel.load('payday_hdpmodel')
student_hdpmodel=HdpModel.load('student_hdpmodel')
mortgage_hdpmodel=HdpModel.load('mortgage_hdpmodel')

In [6]:
#https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_pickle.html
dfmortproddebtcoll_common_cos=pd.read_pickle('dfmortproddebtcoll_common_cos.pickle')
dfpayproddebtcoll_common_cos=pd.read_pickle('dfpayproddebtcoll_common_cos.pickle')
dfstudproddebtcoll_common_cos=pd.read_pickle('dfstudproddebtcoll_common_cos.pickle')

In [7]:
#https://stackoverflow.com/questions/4530611/saving-and-loading-objects-and-using-pickle
payday_filehandler = open('payday_dictionary.pickle','rb')
payday_dictionary = pickle.load(payday_filehandler)
payday_filehandler.close()

student_filehandler = open('student_dictionary.pickle','rb')
student_dictionary = pickle.load(student_filehandler)
student_filehandler.close()

mortgage_filehandler = open('mortgage_dictionary.pickle','rb')
mortgage_dictionary = pickle.load(mortgage_filehandler)
mortgage_filehandler.close()

In [8]:
dfmortproddebtcoll_common_cos.head()

,Date received,Product,Sub-product,Issue,Sub-issue,Consumer complaint narrative,Company public response,Company,State,ZIP code,Tags,Consumer consent provided?,Submitted via,Date sent to company,Company response to consumer,Timely response?,Consumer disputed?,Complaint ID
1,2015-07-16,Mortgage,Conventional fixed mortgage,"Application, originator, mortgage broker",NaN,"In XXXX, I and my ex-husband applied for a ref...",NaN,HSBC NORTH AMERICA HOLDINGS INC.,IL,625XX,NaN,Consent provided,Web,07/16/2015,Closed with explanation,Yes,No,1472017
3,2015-03-24,Mortgage,Conventional fixed mortgage,Settlement process and costs,NaN,I got recent modification ( XXXX/XXXX/2015 ) f...,NaN,OCWEN LOAN SERVICING LLC,TX,774XX,NaN,Consent provided,Web,03/24/2015,Closed with explanation,Yes,No,1297495
4,2015-03-30,Mortgage,Conventional fixed mortgage,Settlement process and costs,NaN,I filed a complain against Chase Mortgage on X...,NaN,JPMORGAN CHASE & CO.,MD,207XX,NaN,Consent provided,Web,03/30/2015,Closed with explanation,Yes,No,1306571
5,2015-07-06,Mortgage,Conventional fixed mortgage,Settlement process and costs,NaN,XXXX previous case numberSince my last written...,NaN,NATIONSTAR MORTGAGE,GA,305XX,NaN,Consent provided,Web,07/06/2015,Closed with explanation,Yes,No,1453603
6,2015-09-29,Mortgage,Conventional fixed mortgage,"Application, originator, mortgage broker",NaN,I received exceptional service from my lender ...,Company chooses not to provide a public response,"SUNTRUST BANKS, INC.",DC,200XX,NaN,Consent provided,Web,10/01/2015,Closed,Yes,No,1584312


In [9]:
dfmortproddebtcoll_common_cos['Date received']

1       2015-07-16
3       2015-03-24
4       2015-03-30
5       2015-07-06
6       2015-09-29
7       2016-12-14
8       2015-04-28
9       2016-01-22
10      2017-01-24
11      2016-04-04
12      2017-01-29
13      2015-08-05
14      2017-03-15
15      2016-02-23
17      2015-06-10
18      2015-12-09
19      2015-06-05
20      2016-08-22
21      2017-03-08
22      2016-12-09
23      2015-04-17
25      2016-08-07
26      2016-05-05
27      2016-12-05
28      2016-09-23
29      2016-07-06
30      2016-10-11
31      2016-11-28
32      2017-03-07
33      2015-09-09
           ...    
15223   2017-04-06
15224   2015-12-31
15225   2017-03-13
15226   2017-02-21
15227   2016-10-17
15229   2016-08-26
15230   2017-02-27
15232   2016-11-16
15233   2015-12-25
15235   2016-10-31
15236   2016-06-21
15237   2016-11-15
15238   2015-12-11
15239   2015-07-01
15240   2016-06-23
15241   2016-09-30
15242   2016-06-10
15243   2016-10-14
15244   2017-04-19
15245   2016-12-02
15246   2016-08-15
15248   2016

In [10]:
k=2008*12+9 # September 2008
period=4 # Quarter year
dfmortproddebtcoll_common_cos['Quarter']=dfmortproddebtcoll_common_cos['Date received'].apply(lambda x: int((x.year*12+x.month-k)/period))
dfpayproddebtcoll_common_cos['Quarter']=dfpayproddebtcoll_common_cos['Date received'].apply(lambda x: int((x.year*12+x.month-k)/period))
dfstudproddebtcoll_common_cos['Quarter']=dfstudproddebtcoll_common_cos['Date received'].apply(lambda x: int((x.year*12+x.month-k)/period))

In [11]:
#dfstudproddebtcoll_common_cos['Quarter']

In [12]:
import nltk

In [13]:
from nltk.corpus import sentiwordnet as swn 

In [14]:
#https://github.com/bhargavvader/personal/blob/master/notebooks/text_analysis_tutorial/topic_modelling.ipynb
#https://www.youtube.com/watch?v=ZkAFJwi-G98
import spacy
nlp = spacy.load('en_core_web_sm')

In [15]:
#from my ComplaintTopicInterpretationSaveTopicModelCleandfMatrixFactorizationv0_0.ipynb Notebook
my_stop_words=[u'bank',u'payment',u'loan',u'go',u'i',u'from',u'my',u'saying',u'download',u'being',u'info',u'XXXX',u'pay',u'student',u'company',u'account',u'paying',u'loans',u'payments',u'I',u'mortgage',u'the',u'home',u'house',u'property',u'We',u'The',u'XX',u'because',u'Bank',u'America',u'Wells',u'Fargo',u'They',u"n't",u'$',u'time',u'told',u'\n\n',u'  ',u'As',u'money',u'debt',u'he',u'day',u'work',u'said',u'called',u'calls',u'phone',u'number',u'He',u'payday',u'Mae',u'Sallie',u'month',u'debt',u'receive',u'pay',u'go',u'send',u'ask',u'sell',u'borrower',u'facility',u'time',u'year',u'month',u'robo',u'middle',u'check',u'debt',u'daughter',u'week',u'year',u'session',u'pg',u'xxxxpage',u'mis',u'friend',u'credit',u'yard',u'human',u'citi',u'acre',u'someone',u'i.',u'chairman',u'name',u'manner',u'refer',u'corporation',u'grandmother',u'cc',u'cycle',u'people',u'girlfriend',u'onset',u'resource',
u'preciousdriver',u'gal',u'man',u'daughter',u'georgia',u'agi',u'boundrie',u'set',u'thank',u'ed',u'bill',u'significance',u'bless',u'what',u'i',u'k',u'fedloan',u'hotel',u'crew',u'head',u'performer',u'c.',u'bofa.',u'hail',u'agent',u'me.all',u'that:1.1',u'crystal',u'statusdespite',u'layer',u'hr',u'midst',u'absence',u'situation',u'error',u'modification',u'loss!?!?!?!.',u'doing',u'grandmother',u'retirement',u'st',u'husband',u'sir',u'girl',u'what',u'i',u'lady',u'claiming',u'senior',u'colleague',u'birth',u'use',u'drafting',u'sake',u'red',u'path',u'navient',u'yrs',u'xxxxgreatlake',u'ring',u'thousand',u'dog',u'd',u'definite',u'minus',u'bofa.',u'co.',u'speaker',u'head',u'difference',u'box',u'sewer',u'spart',u'loan/',u'init',u'service',u'director',u'boss',u'site',u'means',u'est',u'brother',u'hundred',u'kin',u'noon',u'yesterday',u'essential',u'avenue',u'tomorrow',u'sent',u'whom',u'slrp',u'female',u'since',u'brotherinlaws',u'page',u'usefuleness',u'trid',u'purple',u'tid',u'elbow',u'lilac',u'circuit',u'sending',u'abbreviation',u'woman',
u'door',u'name',u'because',u'weekend',u'wk',u'basic',u'thing',u'rank',u'navient',u'someone',u'48hrs',u'dinner',u'it',u'me',u'progress',u'member',u'dismiss'] # v0.5



for stopword in my_stop_words:
    lexeme = nlp.vocab[stopword]
    lexeme.is_stop = True

In [87]:
topic_model_pos_list=['NOUN']
def payday_topic150scores_and_senti(raw_complaint):
    preproc_text=[]
    preproc_sent=[]
    doc=nlp(raw_complaint)
    for token in doc:
        if token.text!='\n':
            if not(token.is_stop) and not(token.is_punct) and token.pos_ in topic_model_pos_list:
                preproc_sent.append(token.lemma_)
        else:
            preproc_text.append(preproc_sent)
            preproc_sent=[]

    preproc_text.append(preproc_sent) #last sentence
    #print(preproc_text)
    
    #https://radimrehurek.com/gensim/models/ldamodel.html
    corpus = [payday_dictionary.doc2bow(text) for text in preproc_text]
    unseen_or_seen_doc = corpus[0]
    
    #print(payday_hdpmodel[unseen_or_seen_doc])

    #based on my I590 Data Science On Ramp Assignment
    positive_score,negative_score=0,0
    for token in doc:
        try:
            if not(token.is_stop) and not(token.is_punct) and token.text!=' ':
                #print(token.lemma_)
                #print(list(swn.senti_synsets(token.lemma_))[0].pos_score())
                #print(list(swn.senti_synsets(token.lemma_))[0].neg_score())
                positive_score+=list(swn.senti_synsets(token.lemma_))[0].pos_score()
                negative_score+=list(swn.senti_synsets(token.lemma_))[0].neg_score()
        except:
            pass
    sentiment_score=positive_score-negative_score
    #print('sentiment is:',sentiment_score)
    senti_topic_output=[0]*151 # one element for sentiment and 150 elements for topics
    for index,prob in payday_hdpmodel[unseen_or_seen_doc]:
        senti_topic_output[index]=prob
    senti_topic_output[150]=sentiment_score
    return senti_topic_output

In [106]:
topic_model_pos_list=['NOUN']
def student_topic150scores_and_senti(raw_complaint):
    preproc_text=[]
    preproc_sent=[]
    doc=nlp(raw_complaint)
    for token in doc:
        if token.text!='\n':
            if not(token.is_stop) and not(token.is_punct) and token.pos_ in topic_model_pos_list:
                preproc_sent.append(token.lemma_)
        else:
            preproc_text.append(preproc_sent)
            preproc_sent=[]

    preproc_text.append(preproc_sent) #last sentence
    #print(preproc_text)
    
    #https://radimrehurek.com/gensim/models/ldamodel.html
    corpus = [student_dictionary.doc2bow(text) for text in preproc_text]
    unseen_or_seen_doc = corpus[0]
    
    #print(student_hdpmodel[unseen_or_seen_doc])

    #based on my I590 Data Science On Ramp Assignment
    positive_score,negative_score=0,0
    for token in doc:
        try:
            if not(token.is_stop) and not(token.is_punct) and token.text!=' ':
                #print(token.lemma_)
                #print(list(swn.senti_synsets(token.lemma_))[0].pos_score())
                #print(list(swn.senti_synsets(token.lemma_))[0].neg_score())
                positive_score+=list(swn.senti_synsets(token.lemma_))[0].pos_score()
                negative_score+=list(swn.senti_synsets(token.lemma_))[0].neg_score()
        except:
            pass
    sentiment_score=positive_score-negative_score
    #print('sentiment is:',sentiment_score)
    senti_topic_output=[0]*151 # one element for sentiment and 150 elements for topics
    for index,prob in student_hdpmodel[unseen_or_seen_doc]:
        senti_topic_output[index]=prob
    senti_topic_output[150]=sentiment_score
    return senti_topic_output

In [107]:
topic_model_pos_list=['NOUN']
def mortgage_topic150scores_and_senti(raw_complaint):
    preproc_text=[]
    preproc_sent=[]
    doc=nlp(raw_complaint)
    for token in doc:
        if token.text!='\n':
            if not(token.is_stop) and not(token.is_punct) and token.pos_ in topic_model_pos_list:
                preproc_sent.append(token.lemma_)
        else:
            preproc_text.append(preproc_sent)
            preproc_sent=[]

    preproc_text.append(preproc_sent) #last sentence
    #print(preproc_text)
    
    #https://radimrehurek.com/gensim/models/ldamodel.html
    corpus = [mortgage_dictionary.doc2bow(text) for text in preproc_text]
    unseen_or_seen_doc = corpus[0]
    
    #print(mortgage_hdpmodel[unseen_or_seen_doc])

    #based on my I590 Data Science On Ramp Assignment
    positive_score,negative_score=0,0
    for token in doc:
        try:
            if not(token.is_stop) and not(token.is_punct) and token.text!=' ':
                #print(token.lemma_)
                #print(list(swn.senti_synsets(token.lemma_))[0].pos_score())
                #print(list(swn.senti_synsets(token.lemma_))[0].neg_score())
                positive_score+=list(swn.senti_synsets(token.lemma_))[0].pos_score()
                negative_score+=list(swn.senti_synsets(token.lemma_))[0].neg_score()
        except:
            pass
    sentiment_score=positive_score-negative_score
    #print('sentiment is:',sentiment_score)
    senti_topic_output=[0]*151 # one element for sentiment and 150 elements for topics
    for index,prob in mortgage_hdpmodel[unseen_or_seen_doc]:
        senti_topic_output[index]=prob
    senti_topic_output[150]=sentiment_score
    return senti_topic_output

In [88]:
#payday_topic150scores_and_senti('Hell pain Have been cheated of all my money will take the bank to court and make them pay This is horrible')

In [89]:
#payday_topic150scores_and_senti(r'@SassyArmylover EN ESTE TIEMPO QUE TE CONOZCO QUIERO DECIR QUE ERES UNA DE LAS PERSONAS QUE MAS APRECIO TE TOMAS E… https://t.co/4GIE7QlU9P')

In [108]:
#tiny_dfpayproddebtcoll_common_cos=dfpayproddebtcoll_common_cos[:100]

In [109]:
#tiny_dfpayproddebtcoll_common_cos.shape

In [110]:
#tiny_dfpayproddebtcoll_common_cos['Sentiment']=tiny_dfpayproddebtcoll_common_cos['Consumer complaint narrative'].apply(lambda x: payday_topic150scores_and_senti(x))

In [111]:
#tiny_dfpayproddebtcoll_common_cos['Sentiment']

In [112]:
#df_temp=pd.concat([tiny_dfpayproddebtcoll_common_cos['Sentiment'],pd.DataFrame(tiny_dfpayproddebtcoll_common_cos['Sentiment'].values.tolist(), index= tiny_dfpayproddebtcoll_common_cos['Sentiment'].index)],axis=1)

In [99]:
dfpayproddebtcoll_common_cos['Sentiment']=dfpayproddebtcoll_common_cos['Consumer complaint narrative'].apply(lambda x: payday_topic150scores_and_senti(x))

In [100]:
df_temp=pd.concat([dfpayproddebtcoll_common_cos['Sentiment'],pd.DataFrame(dfpayproddebtcoll_common_cos['Sentiment'].values.tolist(), index= dfpayproddebtcoll_common_cos['Sentiment'].index)],axis=1)

In [102]:
dfpayproddebtcoll_common_cos.drop(['Sentiment'],axis=1,inplace=True)

In [103]:
df_temp.drop(['Sentiment'],axis=1,inplace=True)

In [104]:
dfpayproddebtcoll_common_cos_ratings=pd.concat([dfpayproddebtcoll_common_cos,df_temp],axis=1)

In [105]:
dfpayproddebtcoll_common_cos_ratings

,Date received,Product,Sub-product,Issue,Sub-issue,Consumer complaint narrative,Company public response,Company,State,ZIP code,...,141,142,143,144,145,146,147,148,149,150
0,2016-12-03,Payday loan,NaN,Charged bank acct wrong day or amt,NaN,Collections agency was giving the wrong inform...,NaN,Ad Astra Recovery Services Inc,LA,707XX,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,-1.875
1,2015-09-02,Payday loan,NaN,Charged fees or interest I didn't expect,NaN,They Keep HARASSING me using fake numbers from...,NaN,ADR ARBITRATION SERVICES (Closed),TX,760XX,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,-1.750
3,2016-01-31,Payday loan,NaN,Charged fees or interest I didn't expect,NaN,I TOOK OUT A PAYDAY LOAN WITH ADVANCE FINANCIA...,Company believes it acted appropriately as aut...,"Harpeth Financial Services, LLC",TN,370XX,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.257728,0.0,-2.875
4,2016-04-27,Payday loan,NaN,Can't stop charges to bank account,NaN,"Evergreen Financial Investments , Inc DBA Rapi...",NaN,CURO Intermediate Holdings,WA,986XX,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,-0.125
5,2016-02-25,Payday loan,NaN,Received a loan I didn't apply for,NaN,They saying I had got a pay day loan but I did...,Company believes complaint caused principally ...,"Enova International, Inc.",AL,352XX,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000
6,2015-08-26,Payday loan,NaN,Charged fees or interest I didn't expect,NaN,"The loan was for XXXX, I was charged XXXX mont...",Company believes it acted appropriately as aut...,"LDF Holdings, LLC",PA,179XX,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000
7,2016-02-23,Payday loan,NaN,Received a loan I didn't apply for,NaN,"On Sunday XXXX XXXX, 2016 I was called by Cash...",NaN,"COMMUNITY CHOICE FINANCIAL, INC.",TX,752XX,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,1.250
8,2016-09-27,Payday loan,NaN,Charged fees or interest I didn't expect,NaN,hi. i have loaned money from ace cash express ...,NaN,"ACE CASH EXPRESS, INC.",MN,551XX,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.625
9,2017-01-04,Payday loan,NaN,Received a loan I didn't apply for,NaN,My identity was stolen in XX/XX/2009. A XXXX X...,NaN,"Mobiloans, LLC",FL,331XX,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.625
10,2015-04-02,Payday loan,NaN,Charged fees or interest I didn't expect,NaN,I was in a bind and took out a loan with Castl...,NaN,"Big Picture Loans, LLC",OK,746XX,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,1.125


In [116]:
dfstudproddebtcoll_common_cos['Sentiment']=dfstudproddebtcoll_common_cos['Consumer complaint narrative'].apply(lambda x: student_topic150scores_and_senti(x))
df_temp=pd.concat([dfstudproddebtcoll_common_cos['Sentiment'],pd.DataFrame(dfstudproddebtcoll_common_cos['Sentiment'].values.tolist(), index= dfstudproddebtcoll_common_cos['Sentiment'].index)],axis=1)
dfstudproddebtcoll_common_cos.drop(['Sentiment'],axis=1,inplace=True)
df_temp.drop(['Sentiment'],axis=1,inplace=True)
dfstudproddebtcoll_common_cos_ratings=pd.concat([dfstudproddebtcoll_common_cos,df_temp],axis=1)
dfstudproddebtcoll_common_cos_ratings

,Date received,Product,Sub-product,Issue,Sub-issue,Consumer complaint narrative,Company public response,Company,State,ZIP code,...,141,142,143,144,145,146,147,148,149,150
0,2017-02-06,Student loan,Federal student loan servicing,Dealing with my lender or servicer,Don't agree with fees charged,I took out a student loan back in XXXX in the ...,Company believes it acted appropriately as aut...,Texas Guaranteed,TX,750XX,...,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.000000,4.125
1,2016-12-07,Student loan,Federal student loan servicing,Dealing with my lender or servicer,Received bad information about my loan,My lender is Nelnet for Federal student loans....,NaN,"NELNET, INC.",NC,282XX,...,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.000000,0.000
2,2016-09-21,Student loan,Federal student loan servicing,Dealing with my lender or servicer,Don't agree with fees charged,I have had a federal student loan since I star...,NaN,AES/PHEAA,MD,210XX,...,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.000000,1.250
3,2017-03-04,Student loan,Federal student loan servicing,Dealing with my lender or servicer,Received bad information about my loan,I believe I was wronged by Navient in several ...,NaN,"Navient Solutions, LLC.",CA,926XX,...,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.000000,-1.125
4,2017-02-11,Student loan,Federal student loan servicing,Dealing with my lender or servicer,Need information about my balance/terms,I 'm dealing with Navient and it has been a ha...,NaN,"Navient Solutions, LLC.",AR,720XX,...,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.000000,2.875
5,2017-01-20,Student loan,Federal student loan servicing,Dealing with my lender or servicer,Keep getting calls about my loan,I tried to make payments that I could afford b...,NaN,"Navient Solutions, LLC.",VA,221XX,...,0.0,0.0,0.265029,0.000000,0.0,0.0,0.0,0.0,0.000000,0.500
6,2017-02-10,Student loan,Federal student loan servicing,Can't repay my loan,Can't decrease my monthly payments,The company does not work with me. I have been...,NaN,AES/PHEAA,TX,751XX,...,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.000000,1.375
7,2016-04-25,Student loan,Federal student loan servicing,Dealing with my lender or servicer,Received bad information about my loan,I borrowed over much time from XXXX ( my paren...,NaN,AES/PHEAA,NY,117XX,...,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.000000,3.625
8,2017-02-28,Student loan,Federal student loan servicing,Dealing with my lender or servicer,Keep getting calls about my loan,The loan provider took me out of deferment whi...,NaN,"Navient Solutions, LLC.",NC,282XX,...,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.000000,0.000
9,2016-10-23,Student loan,Federal student loan servicing,Dealing with my lender or servicer,Having problems with customer service,I have a breach of contract borrowers defense ...,NaN,"Navient Solutions, LLC.",NV,891XX,...,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.000000,1.625


In [117]:
dfpayproddebtcoll_common_cos_ratings.to_pickle('dfpayproddebtcoll_common_cos_ratings.pickle')

In [118]:
dfstudproddebtcoll_common_cos_ratings.to_pickle('dfstudproddebtcoll_common_cos_ratings.pickle')

In [119]:
dfmortproddebtcoll_common_cos['Sentiment']=dfmortproddebtcoll_common_cos['Consumer complaint narrative'].apply(lambda x: mortgage_topic150scores_and_senti(x))
df_temp=pd.concat([dfmortproddebtcoll_common_cos['Sentiment'],pd.DataFrame(dfmortproddebtcoll_common_cos['Sentiment'].values.tolist(), index= dfmortproddebtcoll_common_cos['Sentiment'].index)],axis=1)
dfmortproddebtcoll_common_cos.drop(['Sentiment'],axis=1,inplace=True)
df_temp.drop(['Sentiment'],axis=1,inplace=True)
dfmortproddebtcoll_common_cos_ratings=pd.concat([dfmortproddebtcoll_common_cos,df_temp],axis=1)
dfmortproddebtcoll_common_cos_ratings

,Date received,Product,Sub-product,Issue,Sub-issue,Consumer complaint narrative,Company public response,Company,State,ZIP code,...,141,142,143,144,145,146,147,148,149,150
1,2015-07-16,Mortgage,Conventional fixed mortgage,"Application, originator, mortgage broker",NaN,"In XXXX, I and my ex-husband applied for a ref...",NaN,HSBC NORTH AMERICA HOLDINGS INC.,IL,625XX,...,0.000000,0.000000,0.0,0.0,0.00000,0.0,0.0,0.000000,0.0,1.875
3,2015-03-24,Mortgage,Conventional fixed mortgage,Settlement process and costs,NaN,I got recent modification ( XXXX/XXXX/2015 ) f...,NaN,OCWEN LOAN SERVICING LLC,TX,774XX,...,0.000000,0.000000,0.0,0.0,0.00000,0.0,0.0,0.000000,0.0,2.000
4,2015-03-30,Mortgage,Conventional fixed mortgage,Settlement process and costs,NaN,I filed a complain against Chase Mortgage on X...,NaN,JPMORGAN CHASE & CO.,MD,207XX,...,0.000000,0.000000,0.0,0.0,0.00000,0.0,0.0,0.000000,0.0,0.625
5,2015-07-06,Mortgage,Conventional fixed mortgage,Settlement process and costs,NaN,XXXX previous case numberSince my last written...,NaN,NATIONSTAR MORTGAGE,GA,305XX,...,0.000000,0.000000,0.0,0.0,0.00000,0.0,0.0,0.000000,0.0,0.125
6,2015-09-29,Mortgage,Conventional fixed mortgage,"Application, originator, mortgage broker",NaN,I received exceptional service from my lender ...,Company chooses not to provide a public response,"SUNTRUST BANKS, INC.",DC,200XX,...,0.000000,0.000000,0.0,0.0,0.00000,0.0,0.0,0.000000,0.0,0.500
7,2016-12-14,Mortgage,Conventional fixed mortgage,"Loan modification,collection,foreclosure",NaN,This loan closed as a short sale with XXXX XXX...,NaN,Selene Finance LP,WA,984XX,...,0.000000,0.000000,0.0,0.0,0.00000,0.0,0.0,0.000000,0.0,0.625
8,2015-04-28,Mortgage,Conventional fixed mortgage,"Loan servicing, payments, escrow account",NaN,( To preface this may be nothing but it is fro...,NaN,"Seterus, Inc.",MP,486XX,...,0.000000,0.000000,0.0,0.0,0.00000,0.0,0.0,0.000000,0.0,1.375
9,2016-01-22,Mortgage,Conventional fixed mortgage,"Loan modification,collection,foreclosure",NaN,In XXXX XXXX my wife and I applied through the...,Company believes it acted appropriately as aut...,"SELECT PORTFOLIO SERVICING, INC.",NaN,NaN,...,0.000000,0.000000,0.0,0.0,0.00000,0.0,0.0,0.000000,0.0,0.625
10,2017-01-24,Mortgage,Conventional fixed mortgage,"Loan modification,collection,foreclosure",NaN,Submitted a short sale package to Loan Care vi...,Company has responded to the consumer and the ...,"LoanCare, LLC",FL,330XX,...,0.000000,0.000000,0.0,0.0,0.00000,0.0,0.0,0.000000,0.0,4.500
11,2016-04-04,Mortgage,Conventional fixed mortgage,"Loan servicing, payments, escrow account",NaN,the day after I closed on a new loan it was so...,Company believes complaint is the result of an...,"Shellpoint Partners, LLC",TX,786XX,...,0.000000,0.000000,0.0,0.0,0.00000,0.0,0.0,0.000000,0.0,0.125


In [120]:
dfmortproddebtcoll_common_cos_ratings.to_pickle('dfmortproddebtcoll_common_cos_ratings.pickle')

In [122]:
dfmortproddebtcoll_common_cos_ratings.columns

Index([               'Date received',                      'Product',
                        'Sub-product',                        'Issue',
                          'Sub-issue', 'Consumer complaint narrative',
            'Company public response',                      'Company',
                              'State',                     'ZIP code',
       ...
                                  141,                            142,
                                  143,                            144,
                                  145,                            146,
                                  147,                            148,
                                  149,                            150],
      dtype='object', length=170)

# See the pickle files generated!